In [1]:
import numpy as np 
import pandas as pd 
import os
import os
import zipfile
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

In [2]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Training')

In [3]:
parent_dir="./Training"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [4]:
save_path = './Training/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)
save_path = './Training/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)

In [5]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Validation')

In [6]:
parent_dir="./Validation"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [7]:
save_path = './Validation/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/'+filename,save_path)
save_path = './Validation/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/'+filename,save_path)

In [8]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Testing')

In [9]:
parent_dir="./Testing"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [10]:
save_path = './Testing/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Test_Set/Test_Set/Test/'+filename,save_path)
save_path = './Testing/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Test_Set/Test_Set/Test/'+filename,save_path)

In [11]:
train_abnormal_dir = os.path.join('./Training/Abnormal')
train_normal_dir = os.path.join('./Training/Normal')
train_abnormal_names = os.listdir(train_abnormal_dir)
train_normal_names = os.listdir(train_normal_dir)
print('total training images with abnormalities:', len(os.listdir(train_abnormal_dir)))
print('total training images of normal eyes:', len(os.listdir(train_normal_dir)))

total training images with abnormalities: 1519
total training images of normal eyes: 401


In [12]:
validation_abnormal_dir = os.path.join('./Validation/Abnormal')
validation_normal_dir = os.path.join('./Validation/Normal')
validation_abnormal_names = os.listdir(validation_abnormal_dir)
validation_normal_names = os.listdir(validation_normal_dir)
print('total validation images with abnormalities:', len(os.listdir(validation_abnormal_dir)))
print('total validation images of normal eyes:', len(os.listdir(validation_normal_dir)))

total validation images with abnormalities: 506
total validation images of normal eyes: 134


In [13]:
test_abnormal_dir = os.path.join('./Testing/Abnormal')
test_normal_dir = os.path.join('./Testing/Normal')
test_abnormal_names = os.listdir(test_abnormal_dir)
test_normal_names = os.listdir(test_normal_dir)
print('total testing images with abnormalities:', len(os.listdir(test_abnormal_dir)))
print('total testing images of normal eyes:', len(os.listdir(test_normal_dir)))

total testing images with abnormalities: 506
total testing images of normal eyes: 134


In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 30,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    zoom_range = 0.1,
                                    horizontal_flip = True
                                   )
train_generator = train_datagen.flow_from_directory('./Training', 
                                                    target_size=(150,150), 
                                                    class_mode='binary')

Found 1920 images belonging to 2 classes.


In [15]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory('./Validation', 
                                                              target_size=(150,150),
                                                              class_mode='binary')

Found 640 images belonging to 2 classes.


In [16]:
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory('./Testing', 
                                                              target_size=(150,150),
                                                              class_mode='binary')

Found 640 images belonging to 2 classes.


In [17]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

In [21]:
model = EfficientNetB0(input_shape=(150,150,3), weights='imagenet', include_top=False)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

16719872/16705208 [==============================] - 0s 0us/step


In [22]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [23]:
for layer in model.layers:
    layer.trainable = False

In [24]:
x = model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

In [25]:
m = Model(inputs=model.input, outputs=predictions)
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [26]:
m.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [27]:
results = m.fit(train_generator,epochs=10,
                              steps_per_epoch=20,
                              validation_data=validation_generator,
                             validation_steps=5)

2022-01-16 05:18:13.806483: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
20/20 [==============================] - 137s 7s/step - loss: 1.3806 - accuracy: 0.7625 - val_loss: 0.8001 - val_accuracy: 0.2250
Epoch 2/10
20/20 [==============================] - 120s 6s/step - loss: 0.6029 - accuracy: 0.7578 - val_loss: 0.4977 - val_accuracy: 0.8062
Epoch 3/10
20/20 [==============================] - 121s 6s/step - loss: 0.5248 - accuracy: 0.8062 - val_loss: 0.5469 - val_accuracy: 0.7812
Epoch 4/10
20/20 [==============================] - 121s 6s/step - loss: 0.5988 - accuracy: 0.7672 - val_loss: 0.7178 - val_accuracy: 0.7375
Epoch 5/10
20/20 [==============================] - 114s 6s/step - loss: 0.5688 - accuracy: 0.7812 - val_loss: 0.5173 - val_accuracy: 0.7875
Epoch 6/10
20/20 [==============================] - 118s 6s/step - loss: 0.5558 - accuracy: 0.7937 - val_loss: 0.5372 - val_accuracy: 0.7812
Epoch 7/10
20/20 [==============================] - 113s 6s/step - loss: 0.5334 - accuracy: 0.8016 - val_loss: 0.6437 - val_accuracy: 0.7375
Epoch 8/10
20

In [28]:
m.evaluate(test_generator)

20/20 [==============================] - 102s 5s/step - loss: 0.5205 - accuracy: 0.7906


[0.5205336213111877, 0.7906249761581421]